<a href="https://colab.research.google.com/github/VihaanChhabria/congress-llm/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install openai-whisper
!pip install pyannote-audio
!pip install torchvision==0.23.0 --no-dependencies

  Using cached numpy-2.4.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)
Using cached numpy-2.4.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompatible.
torchvision 0.24.0+cu126 requires torch==2.9.0, but you have torch 2.8.0 which is incompatible.
opencv-contrib-python 4.12.0.88 re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 126.2 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.24.0+cu126
    Uninstalling torchvision-0.24.0+cu126:
      Successfully uninstalled torchvision-0.24.0+cu126


In [ ]:
audio_path = "/content/drive/My Drive/Colab Notebooks/audio.m4a"

In [ ]:
##### transcription setup #####

import whisper
import os

model = whisper.load_model("tiny.en")
result = model.transcribe(audio_path, verbose=True)

# Each segment has start, end, and text
for segment in result['segments']:
    print(segment['start'], segment['end'], segment['text'])

100%|█████████████████████████████████████| 72.1M/72.1M [00:01<00:00, 46.8MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:06.720]  They can spend more money on enrichment activities, extra-curriculars, rich high school
[00:06.720 --> 00:12.320]  dropouts, staying at the top as much as poor college graduates stay in the bottom from 14-16%.
[00:12.320 --> 00:17.880]  It gets worse when you factor in race, as historical divides and wealth created by decades
[00:17.880 --> 00:23.320]  of redlining, slavery, Jim Crow laws, Chinese and Sluge and the literal internment camps
[00:23.320 --> 00:25.640]  that we put Japanese people in.
[00:25.640 --> 00:30.400]  More aspirants who only want to pursue the American dream, they were promised or instead
[00:30.400 --> 00:34.800]  burdened with college debts that keep them held under, drowning in the river of lies promised
[00:34.800 --> 00:36.800]  by our founding fathers.
[00:36.800 --> 00:39.160]  Blind hiring sounds like a good thing.
[00:39.160 --> 00:40.160]  Keyword sounds.
[00:40.160 --> 00:42.280]  Let's do a quick case study.
[00:42.280 --> 0

In [ ]:
#### diarization setup ####

import os
import torch
import torchaudio
from pyannote.audio import Pipeline
from pyannote.audio.pipelines.utils.hook import ProgressHook

from google.colab import userdata

# Define a path for the converted WAV file
converted_audio_path = "/content/drive/My Drive/Colab Notebooks/audio_converted.wav"

# Load the M4A file and save it as WAV using torchaudio
print(f"Converting {audio_path} to WAV...")
waveform, sample_rate = torchaudio.load(audio_path)
torchaudio.save(converted_audio_path, waveform, sample_rate)
print(f"Conversion complete. Saved to {converted_audio_path}")

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-community-1",
    token=userdata.get('pyannoteKey'))

pipeline.to(torch.device("cuda"))

print("Processing converted audio.wav")

with ProgressHook() as hook:
    output = pipeline(converted_audio_path, hook=hook)

for turn, speaker in output.speaker_diarization:
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")

Converting /content/drive/My Drive/Colab Notebooks/audio.m4a to WAV...
Conversion complete. Saved to /content/drive/My Drive/Colab Notebooks/audio_converted.wav


Output()

Processing converted audio.wav


/usr/local/lib/python3.12/dist-packages/pyannote/audio/models/blocks/pooling.py:103: UserWarning: std(): degrees of
freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output 
numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)

start=0.0s stop=1.5s speaker_SPEAKER_03
start=1.8s stop=12.7s speaker_SPEAKER_03
start=6.0s stop=6.2s speaker_SPEAKER_02
start=13.1s stop=24.7s speaker_SPEAKER_03
start=25.5s stop=36.1s speaker_SPEAKER_03
start=36.6s stop=65.4s speaker_SPEAKER_03
start=65.6s stop=71.9s speaker_SPEAKER_03
start=72.4s stop=74.3s speaker_SPEAKER_03
start=74.5s stop=86.5s speaker_SPEAKER_03
start=87.8s stop=90.3s speaker_SPEAKER_03
start=90.6s stop=97.7s speaker_SPEAKER_03
start=98.3s stop=102.1s speaker_SPEAKER_03
start=102.5s stop=107.7s speaker_SPEAKER_03
start=108.5s stop=113.9s speaker_SPEAKER_03
start=114.8s stop=128.0s speaker_SPEAKER_03
start=128.9s stop=132.8s speaker_SPEAKER_03
start=133.4s stop=159.0s speaker_SPEAKER_03
start=149.1s stop=149.2s speaker_SPEAKER_00
start=159.0s stop=159.0s speaker_SPEAKER_01
start=160.0s stop=160.1s speaker_SPEAKER_01
start=161.4s stop=170.9s speaker_SPEAKER_01
start=171.0s stop=171.1s speaker_SPEAKER_01
start=171.2s stop=174.0s speaker_SPEAKER_01
start=174.2s sto